In [2]:
import pandas as pd
import json
from pandas import ExcelFile

In [3]:
# Read each sheet into a dataframe
# Capturing just the state (col A) and 2017 and 2018 values
# set the state as index to ease merging

file = '../data/source/ConsDebt2003-2018XStatePerFed.xlsx'

auto_df = pd.read_excel(file, sheet_name='auto', skiprows=3, index_col=None, usecols = "A,P:Q")
auto_df=auto_df.set_index('state')
cc_df = pd.read_excel(file, sheet_name='creditcard', skiprows=3, index_col=None, usecols = "A,P:Q")
cc_df=cc_df.set_index('state')
mortg_df = pd.read_excel(file, sheet_name='mortgage', skiprows=3, index_col=None, usecols = "A,P:Q")
mortg_df=mortg_df.set_index('state')
stloan_df = pd.read_excel(file, sheet_name='studentloan', skiprows=3, index_col=None, usecols = "A,P:Q")
stloan_df=stloan_df.set_index('state')
autod_df = pd.read_excel(file, sheet_name='auto_delinq', skiprows=3, index_col=None, usecols = "A,P:Q")
autod_df=autod_df.set_index('state')
ccd_df = pd.read_excel(file, sheet_name='creditcard_delinq', skiprows=3, index_col=None, usecols = "A,P:Q")
ccd_df=ccd_df.set_index('state')
mortgd_df = pd.read_excel(file, sheet_name='mortgage_delinq', skiprows=3, index_col=None, usecols = "A,P:Q")
mortgd_df=mortgd_df.set_index('state')
stloand_df = pd.read_excel(file, sheet_name='studentloan_delinq', skiprows=3, index_col=None, usecols = "A,P:Q")
stloand_df=stloand_df.set_index('state')

In [4]:
# Average 2017 and 2018 to a new column
auto_df['debt_auto']=0.5*(auto_df['Q4_2017']+auto_df['Q4_2018'])
cc_df['debt_cc']=0.5*(cc_df['Q4_2017']+auto_df['Q4_2018'])
mortg_df['debt_mortg']=0.5*(mortg_df['Q4_2017']+auto_df['Q4_2018'])
stloan_df['debt_stloan']=0.5*(stloan_df['Q4_2017']+auto_df['Q4_2018'])

autod_df['pct_auto_delinq']=0.5*(autod_df['Q4_2017']+autod_df['Q4_2018'])
ccd_df['pct_cc_delinq']=0.5*(ccd_df['Q4_2017']+ccd_df['Q4_2018'])
mortgd_df['pct_mortg_delinq']=0.5*(mortgd_df['Q4_2017']+mortgd_df['Q4_2018'])
stloand_df['pct_stloan_delinq']=0.5*(stloand_df['Q4_2017']+stloand_df['Q4_2018'])

In [5]:
# drop original yearly values
auto_df=auto_df.drop(['Q4_2017','Q4_2018'], axis=1)
cc_df=cc_df.drop(['Q4_2017','Q4_2018'], axis=1)
mortg_df=mortg_df.drop(['Q4_2017','Q4_2018'], axis=1)
stloan_df=stloan_df.drop(['Q4_2017','Q4_2018'], axis=1)
autod_df=autod_df.drop(['Q4_2017','Q4_2018'], axis=1)
ccd_df=ccd_df.drop(['Q4_2017','Q4_2018'], axis=1)
mortgd_df=mortgd_df.drop(['Q4_2017','Q4_2018'], axis=1)
stloand_df=stloand_df.drop(['Q4_2017','Q4_2018'], axis=1)

In [15]:
# Merge all dataframes (by index)

# 2 by 2 - by subject
df1 = pd.merge(auto_df, autod_df, left_index=True, right_index=True)
df2 = pd.merge(cc_df,ccd_df, left_index=True, right_index=True)
df3 = pd.merge(mortg_df,mortgd_df, left_index=True, right_index=True)
df4 = pd.merge(stloan_df,stloand_df, left_index=True, right_index=True)

# Combine all
dfa = pd.merge(df1, df2, left_index=True, right_index=True)
dfb = pd.merge(df3, df4, left_index=True, right_index=True)

debt_df = pd.merge(dfa, dfb, left_index=True, right_index=True)

# Drop Puerto Rico (not a state, no senators) and the last row "allUS" due to some missing data
debt_df = debt_df.drop(["PR","allUS"])

In [16]:
debt_df = debt_df.reset_index()

In [17]:
debt_df.rename(columns={'state':'state_po'}, inplace=True)

In [18]:
debt_df.head()

,state_po,debt_auto,pct_auto_delinq,debt_cc,pct_cc_delinq,debt_mortg,pct_mortg_delinq,debt_stloan,pct_stloan_delinq
0,AK,4965.0,2.350730,4595.0,6.081184,23250.0,0.649275,4330.0,9.855371
1,AL,4870.0,5.951856,3665.0,7.707510,12980.0,1.076603,4785.0,14.279678
2,AR,5310.0,4.681963,3840.0,8.857235,11790.0,1.043419,4775.0,14.303569
3,AZ,5045.0,4.657981,4170.0,9.701708,20440.0,0.761147,5045.0,12.572649
4,CA,4480.0,3.889095,4000.0,8.299773,29610.0,0.644828,4455.0,9.740924


In [14]:
debt_df.to_csv('../data/clean/debt.csv')